# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Abdelmalack Simodo

Nome: Luiz Fernando da Silva Borges

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [58]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
import re

### Antes da etapa abaixo, os tweets foram classificados manualmente usando a interface de classificação desenvolvida

Reunimos os arquivos com os tweets e suas etiquetas de classificação em um único arquivo Excel

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [59]:
#junta os dois arquivos excel em um unico
df_mensagens_etiquetadas_1 = pd.read_excel('metade_k.xlsx') 
df_mensagens_etiquetadas_2 = pd.read_excel('metade_y.xlsx') 

#a coluna foi nomeada manualmente de 'Tweets'
valores1 = df_mensagens_etiquetadas_1[['Tweets','Etiquetas']]
valores2 = df_mensagens_etiquetadas_2[['Tweets','Etiquetas']]

df_tabela_etiquetada = pd.concat([valores1, valores2])
df_tabela_etiquetada.head(7)

,Tweets,Etiquetas
0,"eu só acho, que todo mundo devia sair do whats...",3
1,vim no habibs,3
2,@viscardivitor @mateus_habibs @bruno_ferraz16 ...,3
3,as vezes eu só queria ver uma trocação de porr...,3
4,@mateus_habibs isso q eh foco,3
5,tinha &gt;&gt;&gt;&gt;&gt;calabresa&lt;&lt;&lt...,2
6,"habibs com entrega grátis, estourei",3


Verificamos se existe pelo menos 252 tweets de cada tipo (positivo, negativo e irrelevante)

In [60]:
#verifica se o numero de cada tipo etiquetas ultrapassa ou alcanca o minimo necessario
distribuicao = df_tabela_etiquetada['Etiquetas'].value_counts()
print(distribuicao)

3    653
1    356
2    263
Name: Etiquetas, dtype: int64


Separa o conjunto de tweets positivos, negativos e irrelevantes em dois dataframes: treino e teste. O dataframe de treino possui 2/3 da quantidade total de tweets coletados. O dataframe de teste possui 1/3 da quantidade total de tweets coletados. 

In [61]:
#separar em teste e treino, tendo a mesma razao de porcentagem de cada tipo de etiqueta
def separa_teste_treino(df):
    #define a divisao
    divisao=1/3
    
    #separa os dataframes em dois novos dataframes
    df_teste, df_treino= np.split(df, [int(divisao*len(df))])

    #junta os dfs em uma lista
    novos_dfs = [df_teste,df_treino]
    return novos_dfs

#chama a funcao
novos_dfs=separa_teste_treino(df_tabela_etiquetada)
df_teste = novos_dfs[0]
df_treino = novos_dfs[1]

In [62]:
df_treino

,Tweets,Etiquetas
424,habibs é tao lixo que da nojo,2
425,"ia no drive do habibs, baixava o vidro do carr...",3
426,vou pia no habibs,3
427,já estava indo dormir mh irmã chegou c vrs esf...,1
428,"em 10 minutos pedi pro lucas:\n\npizza, habibs...",3
...,...,...
631,eu comeria habibs dnv tô com vontade kkkkk,1
632,queria umas esfiha do habibs 🤤🤤,1
633,"pedi uma coisa e veio outra, q ódio desse habi...",2
634,pedi comida no habibs e n chego ate agr ka...,2


In [63]:
df_teste

,Tweets,Etiquetas
0,"eu só acho, que todo mundo devia sair do whats...",3
1,vim no habibs,3
2,@viscardivitor @mateus_habibs @bruno_ferraz16 ...,3
3,as vezes eu só queria ver uma trocação de porr...,3
4,@mateus_habibs isso q eh foco,3
...,...,...
419,quem tá c esfiha do habibs no ônibus??????????...,3
420,"não vou mentir, queria tá agora comendo cogume...",1
421,@galas_oficial ah...vc que quiz dizer habibs?,3
422,@mateus_habibs @flavia_azambuja a isa vai fica...,3


## Conta o numero de cada tipo de tweets no dataframe de treino

In [64]:
#conta o numero de vezes que cada etiqueta se repete e coloca numa lista
distribuicao = df_treino['Etiquetas'].value_counts()
lista_dos_numeros_de_cada_tipo_de_etiqueta_treino = distribuicao.to_numpy()
print("1, 2 e 3 no dataframe de treino:", lista_dos_numeros_de_cada_tipo_de_etiqueta_treino)

distribuicao = df_teste['Etiquetas'].value_counts()
lista_dos_numeros_de_cada_tipo_de_etiqueta_teste = distribuicao.to_numpy()
print("1, 2 e 3 no dataframe de teste:", lista_dos_numeros_de_cada_tipo_de_etiqueta_teste)

1, 2 e 3 no dataframe de treino: [354 284 210]
1, 2 e 3 no dataframe de teste: [299  72  53]


Vamos filtrar os dois conjuntos de dados para
    
* Remover pontuações
* Remover links
* Tornar todas as letras minísculas 
* Colocar um espaço antes e um depois de cada emoji
* Substituir mais de um espaçamento entre dois elementos por apenas um

In [65]:
#Remove as pontuações !-.:?;,%&*_+-\/ e links
def limpa_pontuacao_links(tweet_1):
    
    pontuacoes = '[!-.:?;,%&*_+-\/|]'
    padrao = re.compile(pontuacoes)
    tweet_sem_pontuacoes = re.sub(padrao, "", tweet_1)
    
    tweet_sem_links = re.sub("http[^\s]*", "", tweet_sem_pontuacoes)
    
    tweet_sem_links = tweet_sem_links.replace("\n", " ") ################# COMO TIRAR COM REGEX??!!
    
    return tweet_sem_links

#minha_string = minha_string.replace('\n', '')

#faz com que cada emoji tenha um espaço antes e depois dele
def cerca_emoji(tweet_3):
    lista_str_letras=""
    for letra in tweet_3:
        if letra in emoji.UNICODE_EMOJI:
            lista_str_letras = lista_str_letras + " " + letra + " "
        else:
            lista_str_letras = lista_str_letras + letra

    return lista_str_letras

#faz com que todas as letras sejam minúsculas e com apenas um espaço entre duas palavras
def minusculo_espacos(tweet_3):
    
    tweet_minusculo = tweet_3.lower()
        
    tweet_espacos = re.sub(" +"," ", tweet_minusculo)
    
    return tweet_espacos

#aplica as três funções acima de uma vez
def filtro(tweet):
    tweet = limpa_pontuacao_links(tweet)
    tweet = cerca_emoji(tweet)
    tweet = minusculo_espacos(tweet)
    return tweet

Aplica os filtros citados para a coluna de tweets do dataframe de treino e salva em um novo dataframe filtrado

In [66]:
df_treino_filtrado = df_treino
df_treino_filtrado["Tweets"]  = df_treino_filtrado["Tweets"].apply(filtro)
df_treino_filtrado.head(7)

,Tweets,Etiquetas
424,habibs é tao lixo que da nojo,2
425,ia no drive do habibs baixava o vidro do carro...,3
426,vou pia no habibs,3
427,já estava indo dormir mh irmã chegou c vrs esf...,1
428,em 10 minutos pedi pro lucas pizza habibs mioj...,3
429,aí @ragazzo e @ifood eu pedi já tá atrasado te...,2
430,queria tanto comer umas esfirras do habibs 🥺,1


### Aplica os filtros citados para a coluna de tweets do dataframe de teste e salva em um novo dataframe filtrado

In [67]:
df_teste_filtrado = df_teste
df_teste_filtrado["Tweets"]  = df_teste_filtrado["Tweets"].apply(filtro)
df_teste_filtrado.head(7)

,Tweets,Etiquetas
0,eu só acho que todo mundo devia sair do whatsa...,3
1,vim no habibs,3
2,@viscardivitor @mateushabibs @brunoferraz16 am...,3
3,as vezes eu só queria ver uma trocação de porr...,3
4,@mateushabibs isso q eh foco,3
5,tinha gtgtgtgtgtcalabresaltltltltltlt no meu p...,2
6,habibs com entrega grátis estourei,3


Criando as series de tweets positivos, negativos e irrelevantes de treino

In [68]:
filtro_positivo_treino = df_treino_filtrado["Etiquetas"] == 1
filtro_negativo_treino =  df_treino_filtrado["Etiquetas"] == 2
filtro_irrelevante_treino =  df_treino_filtrado["Etiquetas"] == 3

positivo_treino = df_treino_filtrado.loc[filtro_positivo_treino]
negativo_treino = df_treino_filtrado.loc[filtro_negativo_treino]
irrelevante_treino = df_treino_filtrado.loc[filtro_irrelevante_treino]

serie_positivo_treino = positivo_treino["Tweets"]
serie_negativo_treino = negativo_treino["Tweets"]
serie_irrelevante_treino = irrelevante_treino["Tweets"]
serie_todos_tweets_treino = pd.concat([serie_positivo_treino, serie_negativo_treino, serie_irrelevante_treino])

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

Separando as séries de tweets positivos, negativos e irrelantes do conjunto de treino

In [69]:
string_positivo_treino = " ".join(serie_positivo_treino)
serie_positivo_treino = pd.Series(string_positivo_treino.split())

string_negativo_treino = " ".join(serie_negativo_treino)
serie_negativo_treino = pd.Series(string_negativo_treino.split())

string_irrelevante_treino = " ".join(serie_irrelevante_treino)
serie_irrelevante_treino = pd.Series(string_irrelevante_treino.split())

Criando as tabelas de frequências relativas dos tweets positivos, negativos e irrelevantes do conjunto de treino

In [70]:
tabela_relativa_positivo_treino = serie_positivo_treino.value_counts()
tabela_relativa_negativo_treino = serie_negativo_treino.value_counts()
tabela_relativa_irrelevante_treino = serie_irrelevante_treino.value_counts()

Criando o conjunto que vai nos servir como o conjunto de todos os tweets existentes

In [71]:
string_todos_tweets_treino = string_positivo_treino + string_negativo_treino + string_irrelevante_treino
serie_todos_tweets_treino = pd.Series(string_todos_tweets_treino.split())
tabela_relativa_todos_tweets_treino = serie_todos_tweets_treino.value_counts()

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Calculando priors

In [72]:
probP = len(serie_positivo_treino)/len(serie_todos_tweets_treino)
probN = len(serie_negativo_treino)/len(serie_todos_tweets_treino)
probI = len(serie_irrelevante_treino)/len(serie_todos_tweets_treino)

In [73]:
print(probP, probN, probI)
soma = probP+probN+probI
print(soma)

0.2843035343035343 0.2873354123354123 0.42853430353430355
1.0001732501732503


### Calculando termos P(Tweet|Positivo), P(Tweet|Negativo), P(Tweet|Irrelevante) 

In [74]:
#alpha = 1.0
#V = 435000
alpha = 1
V = 435000

In [75]:
def prob_tweet_positivo(tweet_teste_1):
    prob_tweet_positivo = 1            #evita underlow
    for palavra in tweet_teste_1.split():
        try:
            prob_palavra_tweet_positivo = np.log((tabela_relativa_positivo_treino[palavra] + alpha) / \
            (len(string_positivo_treino) + alpha*V))
            prob_tweet_positivo = prob_tweet_positivo + prob_palavra_tweet_positivo
        except:
            prob_palavra_tweet_positivo = np.log(alpha / (len(string_positivo_treino) + alpha*V))
            prob_tweet_positivo = prob_tweet_positivo + prob_palavra_tweet_positivo
             
    return prob_tweet_positivo

In [76]:
tweet = "@michellebelo7 lula o primeiro"

prob_tweet_positivo(tweet)

-47.17413388876251

In [77]:
def prob_tweet_negativo(tweet_teste_2):
    prob_tweet_negativo = 1             #evita underlow
    for palavra in tweet_teste_2.split():
        try:
            prob_palavra_tweet_negativo = np.log((tabela_relativa_negativo_treino[palavra] + alpha) / \
            (len(string_negativo_treino) + alpha*V))
            prob_tweet_negativo = prob_tweet_negativo + prob_palavra_tweet_negativo
        except:
            prob_palavra_tweet_negativo = np.log(alpha / (len(string_negativo_treino) + alpha*V))
            prob_tweet_negativo = prob_tweet_negativo + prob_palavra_tweet_negativo
        
    return prob_tweet_negativo

In [78]:
def prob_tweet_irrelevante(tweet_teste_3):
    prob_tweet_irrelevante = 1             #evita underlow
    for palavra in tweet_teste_3.split():
        try:
            prob_palavra_tweet_irrelevante = np.log((tabela_relativa_irrelevante_treino[palavra] + alpha) /\
            (len(string_irrelevante_treino) + alpha*V))
            prob_tweet_irrelevante = prob_tweet_irrelevante + prob_palavra_tweet_irrelevante 
        except:
            prob_palavra_tweet_irrelevante = np.log(alpha / (len(string_irrelevante_treino) + alpha*V))
            prob_tweet_irrelevante = prob_tweet_irrelevante + prob_palavra_tweet_irrelevante 
                
    return prob_tweet_irrelevante

### Calculando os termos P(Positivo|Tweet), P(Negativo|Tweet), P(Irrelavante|Tweet) 

In [79]:
def maior_prob(tweet_4):
    prob_positivo_dado_tweet = np.log(probP) + prob_tweet_positivo(tweet_4)
    prob_negativo_dado_tweet = np.log(probN) + prob_tweet_negativo(tweet_4)
    prob_irrelevante_dado_tweet = np.log(probI) + prob_tweet_irrelevante(tweet_4)
    
    #print(prob_positivo_dado_tweet)
    #print(prob_negativo_dado_tweet)
    #print(prob_irrelevante_dado_tweet)
    
    if prob_positivo_dado_tweet > prob_negativo_dado_tweet and prob_positivo_dado_tweet > \
    prob_irrelevante_dado_tweet:
        return 1
    if prob_negativo_dado_tweet > prob_positivo_dado_tweet and  prob_negativo_dado_tweet > \
    prob_irrelevante_dado_tweet:
        return 2
    else:
        return 3

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [80]:
df_teste_classificador = df_teste
df_teste_classificador["Classificacao"] = df_teste_classificador["Tweets"].apply(maior_prob)
df_teste_classificador.head(7)

,Tweets,Etiquetas,Classificacao
0,eu só acho que todo mundo devia sair do whatsa...,3,3
1,vim no habibs,3,3
2,@viscardivitor @mateushabibs @brunoferraz16 am...,3,3
3,as vezes eu só queria ver uma trocação de porr...,3,3
4,@mateushabibs isso q eh foco,3,3
5,tinha gtgtgtgtgtcalabresaltltltltltlt no meu p...,2,3
6,habibs com entrega grátis estourei,3,3


In [81]:
pd.crosstab(df_teste_classificador.Etiquetas,df_teste_classificador.Classificacao, normalize="columns")*100

Classificacao,1,2,3
Etiquetas,,,
1,54.347826,0.0,7.236842
2,6.521739,75.0,8.552632
3,39.130435,25.0,84.210526


In [82]:
lista_classificacao = []
lista_etiquetas = []
acertos_1 = 0
acertos_2 = 0
acertos_3 = 0

for n_linha in range(len(df_teste_classificador["Classificacao"])):
    lista_classificacao.append(df_teste_classificador["Classificacao"].iloc[n_linha])
    lista_etiquetas.append(df_teste_classificador["Etiquetas"].iloc[n_linha])
    
    if df_teste_classificador["Classificacao"].iloc[n_linha] == 1 and \
    df_teste_classificador["Etiquetas"].iloc[n_linha] == 1:
        acertos_1 += 1
    elif df_teste_classificador["Classificacao"].iloc[n_linha] == 2 and \
    df_teste_classificador["Etiquetas"].iloc[n_linha] == 2:
        acertos_2 += 1
    elif df_teste_classificador["Classificacao"].iloc[n_linha] == 3 and \
    df_teste_classificador["Etiquetas"].iloc[n_linha] == 3:
        acertos_3 += 1
        
etiquetas_1 = lista_etiquetas.count(1)
etiquetas_2 = lista_etiquetas.count(2)
etiquetas_3 = lista_etiquetas.count(3)

acc_1 = (acertos_1/etiquetas_1) * 100
acc_2 = (acertos_2/etiquetas_2) * 100
acc_3 = (acertos_3/etiquetas_3) *100
        
print("1: {}/{}= {}%".format(acertos_1, etiquetas_1, acc_1))
print("2: {}/{}= {}%".format(acertos_2, etiquetas_2, acc_2))
print("3: {}/{}= {}%".format(acertos_3, etiquetas_3, acc_3))

1: 50/72= 69.44444444444444%
2: 21/53= 39.62264150943396%
3: 256/299= 85.61872909698997%


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**